# Parte 1 - SQL

1. ¿Puedes describirnos el procedimiento que has seguido para cargar los tres ficheros?
¿Puedes compartirnos la definición de la tabla que estás usando para contestar las
preguntas planteadas?
2. Una vez cargados los tres ficheros en tu base de datos, ¿cuántos registros tiene la
tabla? Compártenos la SQL que has usado.
3. Para cada uno de los tres meses con datos ¿podrías decirnos cuál es el trayecto más
corto, más largo y el trayecto medio?, en distancia y en tiempo. Compártenos la
sentencia SQL que has usado.
4. Para cada uno de los tres meses con datos ¿podrías decirnos cuánta es la variación
porcentual en el número de servicios registrados con respecto al mes anterior? Por
ejemplo, en esta pregunta nos gustaría obtener un resultado así. Compártenos la
sentencia SQL que has usado.

# Tools

In [1]:
# Data extraction tools
import requests
from bs4 import BeautifulSoup

#Data wrangling tools
import pandas as pd
import numpy as np

#DataBase tools
import psycopg2
import psycopg2.extras as extras

# Data Extraction

1a) ¿Puedes describirnos el procedimiento que has seguido para cargar los tres ficheros?

In [2]:
year_html = None 

In [3]:
def soup(year):
    
    url = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    
    print(f"Asking for http content from {url}")
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    
    print(f"Filtering by year provided: {year}...")
    global year_html
    year_html = soup.find_all("div", id=f"faq{year}")[0] 
    print(f"html generated")

In [4]:
def single_month_links_df(year, month):
    
    months_available = [elem.get_text(strip=True) for elem in year_html.find_all(['b'])]
    
    print(f"Checking month requested {month} vs months available in the html content...")
    if month not in months_available:
        raise ValueError(f"month provided does not match with available months, check spelling {months_available}")
    
    print(f"Month provided is valid. Filtering by month...")
    filter_by_month = year_html.find(string=f"{month}").parent.findNext('ul')
    links = [elem.a['href'] for elem in filter_by_month.find_all('li')]
    description = [elem.a.text for elem in filter_by_month.find_all('li')]
    
    single_month_links = pd.DataFrame({'month': month, 'links': links, 'description': description})
    
    pd.set_option('display.max_colwidth', None)
    
    return single_month_links

In [5]:
def all_months_links_df(year, *month):
    
    links = pd.DataFrame()
    
    for elem in month:
        links = links.append(single_month_links_df(year, elem), ignore_index=True)
 
    print("Generating csv links table")
    
    return links

In [6]:
def time_lapse(seconds):
    seconds_in_day = 60 * 60 * 24
    seconds_in_hour = 60 * 60
    seconds_in_minute = 60
    
    days = seconds // seconds_in_day
    hours = (seconds - (days * seconds_in_day)) // seconds_in_hour
    minutes = (seconds - (days * seconds_in_day) - (hours * seconds_in_hour)) // seconds_in_minute
    
    return f"{days} days, {hours} hours, {minutes} minutes"

In [7]:
def csv_to_df(year, taxi_color ,*month):
    
    soup(year) ###
    
    print(f"list of months requested: {month}")
    links_table = all_months_links_df(year, *month)
    links_table = links_table[links_table['description'].str.contains(taxi_color)][['month','links']]
    print(links_table['links'])
    
    final_df = pd.DataFrame()
    
    print("Starting parsing process:")
    for index, elem in enumerate(links_table['links']):
        print(f"Parsing csv {elem}...")
        #m =  month[index]
        df = pd.read_csv(elem)
        df.insert(1, 'month', month[index])
        #df['month'] = m
        final_df = final_df.append(df, ignore_index=True)
        print("Parsing completed")
    
    type_object = final_df.select_dtypes(include='object').columns.to_list()
    type_object.remove('store_and_fwd_flag')
    type_object.remove('month')
    
    for elem in type_object:
        final_df[f"{elem}"] = pd.to_datetime(final_df[f"{elem}"], yearfirst=True, format="%Y/%m/%d %H:%M:%S")
    
    final_df['month'] = pd.to_datetime(final_df['month'], format="%B").dt.month  
    
    trip_duration_seconds = (final_df['tpep_dropoff_datetime'] - final_df['tpep_pickup_datetime']).astype('timedelta64[s]').astype('int')
    trip_duration = trip_duration_seconds.apply(time_lapse)
    
    final_df.insert(4, 'trip_duration_seconds', trip_duration_seconds)
    final_df.insert(5, 'trip_duration', trip_duration)
    
    return final_df

In [8]:
#yellowtaxis_2020= csv_to_df(2020, 'Yellow', 'January', 'February', 'March')
#name = "2020_yellow_jan_feb_mar.csv"
#yellowtaxis_2020.to_csv(name, index=False)

# DataBase creation

1b) ¿Puedes compartirnos la definición de la tabla que estás usando para contestar las
preguntas planteadas?

In [9]:
postgres_server_param_dic = {
    "host"      : "127.0.0.1",
    "database"  : "postgres",
    "user"      : "postgres",
    "password"  : "Snowdav3"
}

db_param_dic = {
    "host"      : "127.0.0.1",
    "database"  : "taxis",
    "user"      : "postgres",
    "password"  : "Snowdav3"
}

In [10]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print('Connection succesful')
    
    return conn

In [11]:
def create_database(database_name):
    
    conn = connect(postgres_server_param_dic)
    conn.autocommit = True
    cursor = conn.cursor()
    sql_create_db = f'''CREATE database {database_name};'''
    print(f"Creating database {database_name}...")
    cursor.execute(sql_create_db)
    print(f"Database {database_name} created successfully !")
    conn.close()

In [12]:
def creating_table(table_name: str):
    
    conn = connect(db_param_dic)
    cursor = conn.cursor()
    
    sql =f'''CREATE TABLE {table_name}(
       VendorID float,
       month int,
       tpep_pickup_datetime timestamp,
       tpep_dropoff_datetime timestamp,
       trip_duration_seconds int,
       trip_duration varchar,
       passenger_count float,
       trip_distance float,
       RatecodeID float,
       store_and_fwd_flag varchar,
       PULocationID float,
       DOLocationID float,
       payment_type float,
       fare_amount float,
       extra float,
       mta_tax float,
       tip_amount float,
       tolls_amount float,
       improvement_surcharge float,
       total_amount float,
       congestion_surcharge float
    );'''
    
    print(f"Creating table {table_name}")
    cursor.execute(sql)
    print(f"Table {table_name} created Successfully !")
    conn.commit()
    #cursor.close()
    #print("Cursor object closed")
    conn.rollback()
    #print(f"Closing {conn.info.dbname} database conection")
    #conn.close()
    #print("Conection closed.")

DataBase Insertion

In [13]:
def execute_batch(database_name: str,table_name: str, year: int, taxi_color: str, *month, page_size=100,):
    
    '''
    
    '''

    df = csv_to_df(year, taxi_color, *month)
    
    print("Converting DataFrame rows to tuples")
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    
    create_database(database_name)
    creating_table(table_name)
    
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table_name, cols)
    conn = connect(db_param_dic)
    cursor = conn.cursor()
    try:
        print(f"Inserting values into the {table_name} table at {conn.info.dbname} database. Please wait, this may take a few minutes...")
        extras.execute_batch(cursor, query, tuples, page_size)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print(f"Batch executed succesfully, all dataframe values inserted into the {table_name} table at {conn.info.dbname}")
    
    print("Closing cursor object...")
    cursor.close()
    print("Cursor object closed.")
    print(f"Closing {conn.info.dbname} database conection...")
    conn.close()
    print("database conection closed.")
      

In [14]:
execute_batch('taxis', 'yellowtaxis', 2020, 'Yellow', 'January', 'February', 'March')

Asking for http content from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
Filtering by year provided: 2020...
html generated
list of months requested: ('January', 'February', 'March')
Checking month requested January vs months available in the html content...
Month provided is valid. Filtering by month...
Checking month requested February vs months available in the html content...
Month provided is valid. Filtering by month...
Checking month requested March vs months available in the html content...
Month provided is valid. Filtering by month...
Generating csv links table
0    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv
4    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-02.csv
8    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-03.csv
Name: links, dtype: object
Starting parsing process:
Parsing csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv...


/home/david/miniconda3/envs/work/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Parsing completed
Parsing csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-02.csv...
Parsing completed
Parsing csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-03.csv...
Parsing completed
Converting DataFrame rows to tuples
Connecting to the PostgreSQL database...
Connection succesful
Creating database taxis
Database taxis created successfully !
Connecting to the PostgreSQL database...
Connection succesful
Creating table yellowtaxis
Table yellowtaxis created Successfully !
Connecting to the PostgreSQL database...
Connection succesful
Inserting values into the yellowtaxis table at taxis database. Please wait, this may take a few minutes...
Batch executed succesfully, all dataframe values inserted into the yellowtaxis table at taxis
Closing cursor object...
Cursor object closed.
Closing taxis database conection...
None database conection closed.


In [15]:
raise ValueError("Stop here")

ValueError: Stop here

In [ ]:
############################################################################################################

In [ ]:
############################################################################################################

2. Una vez cargados los tres ficheros en tu base de datos, ¿cuántos registros tiene la
tabla? Compártenos la SQL que has usado.

In [ ]:
registros = '''SELECT COUNT(vendorid) AS registros_totales
FROM yellowtaxis;
'''
conn.rollback()
cursor = conn.cursor()
cursor.execute(registros)
result = cursor.fetchall()[0]
print(f"La tabla yellowtaxis de la database {conn.info.dbname} contiene {result[0]} registros")

3. Para cada uno de los tres meses con datos ¿podrías decirnos cuál es el trayecto más
corto, más largo y el trayecto medio?, en distancia y en tiempo. Compártenos la
sentencia SQL que has usado.

In [ ]:
def execute_sql_query(month: int, trip_metric, trip_measure):
    
    query = None
    if trip_metric == 'longest':
        if trip_measure == 'time':
            longest_trip_time = f'''
                SELECT vendorid, month, tpep_pickup_datetime, tpep_dropoff_datetime, trip_duration_seconds, trip_duration
                FROM yellowtaxis
                WHERE month = '{month}'
                ORDER BY trip_duration_seconds DESC
                LIMIT 1;
            '''
            query = longest_trip_time
        elif trip_measure == 'distance':
            longest_trip_distance = f'''
                SELECT vendorid, month, tpep_pickup_datetime, tpep_dropoff_datetime,
                trip_duration_seconds, trip_duration, trip_distance
                FROM yellowtaxis
                WHERE month = '{month}'
                ORDER BY trip_distance DESC
                LIMIT 1;
            '''
            query = longest_trip_distance
            
    elif trip_metric == 'shortest':
        if trip_measure == 'time':
            shortest_trip_time = f'''
                SELECT *
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0
                ORDER BY trip_duration_seconds ASC,
                trip_distance ASC
                LIMIT 1;

            '''
            query = shortest_trip_time
            
        elif trip_measure == 'distance':
            shortest_trip_distance = f'''
                SELECT *
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0
                ORDER BY trip_distance ASC,
                trip_duration_seconds ASC
                LIMIT 1;

            '''
            query = shortest_trip_distance
    
    elif trip_metric == 'average':
        if trip_measure == 'time':
            avg_trip_time = f'''
                SELECT AVG(trip_duration_seconds) AS trip_duration_avg
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0

            '''
            query = avg_trip_time
            
        elif trip_measure == 'distance':
            avg_trip_distance = f'''
                SELECT AVG(trip_distance) AS trip_distance_avg
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0

            '''
            query =  avg_trip_distance
    
    cursor.execute(query)
    result = cursor.fetchall()[0]
    
    return result

In [ ]:
conn = connect(db_param_dic)

January

In [ ]:
# Longest time
jan_lg_t = execute_sql_query(1, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from January 2020 took: {jan_lg_t[-1]}")
print()

# Longest distance

jan_lg_d = execute_sql_query(1, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from January 2020 was: {jan_lg_d[-1]} miles")
print()

In [ ]:
# Shortest time
jan_st_t = execute_sql_query(1, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from January 2020 took: {jan_st_t[4]} seconds")
print()

# Shortest distance
jan_st_d = execute_sql_query(1, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from January 2020 was: {jan_st_d[7]} miles")
print()

In [ ]:
# Average
jan_avg_t = execute_sql_query(1, 'average', 'time')
print()
print(f"Average yellow taxi trip time from January 2020 was: {time_lapse(jan_avg_t[0])}")
print()

# Average distance
jan_avg_d = execute_sql_query(1, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from January 2020 was: {jan_avg_d[0]} miles")
print()

February

In [ ]:
feb_lg_t = execute_sql_query(2, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from February 2020 took: {feb_lg_t[-1]}")
print()

# Longest distance

feb_lg_d = execute_sql_query(2, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from February 2020 was: {feb_lg_d[-1]} miles")
print()

In [ ]:
# Shortest time
feb_st_t = execute_sql_query(2, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from February 2020 took: {feb_st_t[4]} seconds")
print()

# Shortest distance
feb_st_d = execute_sql_query(2, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from February 2020 was: {feb_st_d[7]} miles")
print()

In [ ]:
# Average
feb_avg_t =  execute_sql_query(2, 'average', 'time')
print()
print(f"Average yellow taxi trip time from February 2020 was: {time_lapse(feb_avg_t[0])}")
print()

# Average distance
feb_avg_d = execute_sql_query(2, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from February 2020 was: {feb_avg_d[0]} miles")
print()

March

In [ ]:
mar_lg_t = execute_sql_query(3, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from March 2020 took: {mar_lg_t[-1]}")
print()

# Longest distance
mar_lg_d = execute_sql_query(3, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from March 2020 was: {mar_lg_d[-1]} miles")
print()

In [ ]:
# Shortest time
mar_st_t = execute_sql_query(3, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from February 2020 took: {jan_st_t[4]} seconds")
print()

# Shortest distance
mar_st_d = execute_sql_query(3, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from March 2020 was: {mar_st_d[7]} miles")
print()

In [ ]:
# Average
mar_avg_t = execute_sql_query(3, 'average', 'time')
print()
print(f"Average yellow taxi trip time from March 2020 was: {time_lapse(mar_avg_t[0])}")
print()

# Average distance
mar_avg_d = execute_sql_query(3, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from March 2020 was: {mar_avg_d[0]} miles")
print()

4. Para cada uno de los tres meses con datos ¿podrías decirnos cuánta es la variación
porcentual en el número de servicios registrados con respecto al mes anterior? Por
ejemplo, en esta pregunta nos gustaría obtener un resultado así. Compártenos la
sentencia SQL que has usado.

In [ ]:
var_jan_feb = '''
WITH variacion AS(
	WITH previous AS(	
		SELECT month, COUNT(vendorid) AS servicios
		FROM yellowtaxis
		WHERE month = '1' OR month = '2'
		GROUP BY month

	)
	SELECT *,
	LAG (servicios, 1) OVER (ORDER BY month) servicios_mes_anterior
	FROM previous
)
SELECT month, servicios, CAST(
	CAST(
		(servicios - servicios_mes_anterior) * 100.00 / servicios_mes_anterior as decimal(4,1)
	) as decimal(4,2)
)
AS "variacion_mes_anterior (%)"
FROM variacion;
'''
conn.rollback()
cursor.execute(var_jan_feb)
result = cursor.fetchall()
pd.DataFrame(result, columns=['mes', 'servicios', 'variacion_mes_anterior (%)'])

In [ ]:
var_feb_mar = '''
WITH variacion AS(
	WITH previous AS(	
		SELECT month, COUNT(vendorid) AS servicios
		FROM yellowtaxis
		WHERE month = '2' OR month = '3'
		GROUP BY month

	)
	SELECT *,
	LAG (servicios, 1) OVER (ORDER BY month) servicios_mes_anterior
	FROM previous
)
SELECT month, servicios, CAST(
	CAST(
		(servicios - servicios_mes_anterior) * 100.00 / servicios_mes_anterior as decimal(4,1)
	) as decimal(4,2)
)
AS "variacion_mes_anterior (%)"
FROM variacion;
'''
conn.rollback()

cursor.execute(var_feb_mar)
result = cursor.fetchall()
pd.DataFrame(result, columns=['mes', 'servicios', 'variacion_mes_anterior (%)'])